fine-tuning a pretrained object detection model

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax" #"tensorflow"
import keras
#NOTE: had problems with a locked connection to kears_hub, possibly after restarting the kernel a couple of times
import keras_hub
import numpy as np
import tensorflow as tf


In [ ]:
import load_statues
num_classes_custom = len(load_statues.class_mapping) #9
print(f'num_classes_custom={num_classes_custom}')

/Volumes/portable2/data/projects/python_venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Volumes/portable2/data/projects/python_venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Volumes/portable2/data/projects/python_venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/fram

bounding box formats

"xyxy"	Boxes defined by (x_min, y_min, x_max, y_max) — corners of the box
"yxyx"	Boxes defined by (y_min, x_min, y_max, x_max) — same as above, but reordered
"xywh"	Boxes defined by (x_center, y_center, width, height) — center + dimensions
"center_xywh"	Used internally for encoding offsets during training

here, using yxyx which is the same as used by the preset directory file image_converter.json


Label Encoding:
Transform your raw bounding box and class label data into the format expected by the RetinaNet's loss function. This involves generating anchor boxes and assigning ground truth boxes to these anchors, as well as creating classification and regression targets.
KerasCV provides RetinaNetLabelEncoder for this purpose.

Data Augmentation (Optional but Recommended):
Apply data augmentation techniques to increase the diversity of your training data and improve model generalization. For object detection, this requires bounding box-aware augmentations like random scaling, cropping, flipping, and color adjustments.
KerasCV's augmentation layers are particularly useful here as they automatically adjust bounding box coordinates.



In [ ]:
task = keras_hub.models.ObjectDetector.from_preset("hf://keras/retinanet_resnet50_fpn_v2_coco")
      

In [ ]:
keras.__version__

#31.56M parameters
#!pip install huggingface_hub
preset = "hf://keras/retinanet_resnet50_fpn_v2_coco"

#Uses FPN features from P5 level, also fine-tuned on COCO at 800×800 resolution
model = keras_hub.models.RetinaNetObjectDetector.from_preset(
    preset=preset, 
    num_classes=num_classes_custom,
    bounding_box_format="yxyx" #same as used by image_converter.json
)

# keras_hub.models.Preprocessor
#preprocessor = keras_hub.models.ObjectDetectorPreprocessor.from_preset(
#    preset,
#)
preprocessor = keras_hub.models.RetinaNetObjectDetectorPreprocessor.from_preset(
    preset
)

# keras_hub.layers.ImageConverter 
#image_convertor = keras_hub.layers.RetinaNetImageConverter.from_preset(preset)
#image_convertor = preprocessor.image_converter

#RetinaNet typically uses focal loss for classification 
#and smooth L1 loss for bounding box regression.
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss={
        "classification_loss": hub.losses.FocalLoss(alpha=0.25, gamma=2.0),
        "box_loss": hub.losses.SmoothL1Loss()
    },
    metrics=["classification_accuracy"] # Add other relevant metrics
)

for layer in model.backbone.layers:
    layer.trainable = False

'''
label_encoder = keras_hub.layers.RetinaNetLabelEncoder(
    anchor_generator=your_anchor_generator,  # or None to use default
    bounding_box_format="xyxy",              # match your dataset format
    positive_threshold=0.5,
    negative_threshold=0.4,
    encoding_format="center_xywh",
    box_variance=[1.0, 1.0, 1.0, 1.0],
    background_class=-1,
    ignore_class=-2
)
{
  "images": Tensor of shape (batch_size, height, width, channels),
  "labels": {
    "boxes": Tensor of shape (batch_size, num_boxes, 4),
    "classes": Tensor of shape (batch_size, num_boxes)
  }
}

x = ...  # Your image or batch of images
y = {
    "boxes": ...,   # Tensor of shape (batch_size, num_boxes, 4)
    "classes": ...  # Tensor of shape (batch_size, num_boxes)
}
'''

#x_processed, y_processed = preprocessor(x, y)

#model.fit(
#    train_dataset,
#    epochs=num_epochs,
#    validation_data=validation_dataset
#)

